In [0]:
'''coded and commented fully by Zili Luo
Student No.20001744
Net Id: 15zl35
'''

'''install some imported packages'''
!pip install tensorflow-gpu==2.0.0rc numpy tqdm --no-cache-dir

In [1]:
'''essencial imports please check tensor version is 2.00-rc0'''
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm
from google.colab import files
from random import shuffle
import csv
import sklearn.metrics

tf.__version__

'2.0.0-rc0'

In [3]:
'''if using colab, use this function to update training data set if run on local machine just ignore this function'''
uploaded = files.upload()

Saving iris_train.txt to iris_train.txt


In [0]:
'''data processing part extract training dataset
I use shuffle to set the tranning list at random order, this helps to improve the performance
Variables:
  datas:numpy.array of numpy.array
  labels:numpy.array of array
'''
total = []
datas = []
labels = []
with open("iris_train.txt") as f:#if using other name of the training dataset please change the name
  for line in f:
    total.append(line.split(","))
shuffle(total)
for item in total:
  datas.append(np.reshape(np.asarray(item[0:-1]),(4,1)))
  label_text = item[-1].replace("\n","")
  labels.append([[0,1][label_text=="Iris-setosa"],[0,1][label_text=="Iris-versicolor"],[0,1][label_text=="Iris-virginica"]])

datas = np.asarray(datas,dtype='float32')
labels = np.asarray(labels)

In [0]:
'''loss function
Input variables:
  pred:tensor with shape(3,1)
        the predict value of the output from model
  target: tensor with shape(3,1)
        the actual value of the output from model
Return variables:
  the error between actual value and the predicted value
'''
def loss(pred,target):
  return target - pred


In [0]:
'''the proceptron model
it is a 4-input with 3-output unit fully connected perceptron network(ANN) model
State variables:
  self.W: the weight tensor of the network with shape(4,3) all set to zeros at initialize
  self.B: the bias tensor of the network unit with shape(3,1) all set to zeros at initialize
  
__Call__:
  Input variables:
    x:tensor with shape(4,1)
      the input from training set
  Return:tensor with shape(3,1)
      the output of sigmoid activate function with the sum of mutiple of input tensor
      and the weight tensor plus unit bias
'''
class Model(object):
  def __init__(self):
    self.W = tf.Variable(tf.zeros([4,3]))
    self.B = tf.Variable(tf.zeros([3]))
    
   
  def __call__(self,x):
    return tf.sigmoid(tf.math.reduce_sum(self.W * x+ self.B,[0])) #  + self.B

'''the pocket model
State variables:
  self.W: the weight tensor of the pocket with shape(4,3)
  self.B: the bias tensor of the pocket unit with shape(3,1)
  self.run_W: cunnt of the cueeent in-pocket weight tensor's best length of run
  self.run_w: cunnt of the cueeent model weight tensor's best length of run
  self.run_succ_W: count of the current in-pocket weight tensor's total correct prediction on the whole traning set
  
'''
class Pocket(object):
  def __init__(self,model):
    self.W = model.W
    self.B = model.B
    self.run_W = 0
    self.run_w = 0
    self.run_succ_W = 0

    
'''initialize the models'''    
model = Model()
pocket = Pocket(model)

In [0]:
'''training function
this function is a implementation of pocket algrothim with Ratchet to improve overall performance
and set the threshold of Perceptron to fire at 0.5 but in eval function or actual use, I use argmax to get the output.
Since within serval experiments, argmax perform very bad in training, but if use threshold to get the actrual output,
It will also happens more than one perceptron fires at the same time.

Input Variables:
  model: Object
        the ANN model
  pocket:Object
        the pocket model
  inputs:numpy.array of numpy.array
        the whole input dataset on the training data, since I use Ratchet
  output:numpy.array of numpy.array
        the whole actual output dataset of the training dataset.
  index:current index of the training data
  lr: learning rate of current epoch

Return:
  None
'''
def train(model,pocket,inputs,outputs,index,lr):
  value = model(inputs[index])
  a = np.array([1 if value[i] > 0.5 else 0 for i in range(3)])
  current_loss = loss(a,outputs[index])
  if np.array_equal(current_loss,np.array([0,0,0])): #if the the weight predicted right keep going
    pocket.run_w = pocket.run_w + 1
  else:
    if pocket.run_w > pocket.run_W: #when the weight pridected wrong but goes further than the one in the pocket
      run_succ_w = 0                #doing the Ratchet to check if place it in the pocket
      for i in range(len(inputs)):
        test_val = np.array([1 if model(inputs[i])[j] > 0.5 else 0 for j in range(3)])
        if np.array_equal(loss(test_val,outputs[i]),np.array([0,0,0])) == 0:
          run_succ_w = run_succ_w + 1
      if run_succ_w > pocket.run_succ_W: #Ratchet succeed
        pocket.W = model.W
        pocket.B = model.B
        pocket.run_succ_W = run_succ_w
        pocket.run_W = pocket.run_w
    model.W = model.W + lr*current_loss*inputs[index]
    model.B = model.B + lr*current_loss*inputs[index]
    pocket.run_w = 0

In [0]:
'''training steps
just some codes to call the training function
I set the learning rate to 0.2 at initial and mutiple 0.99 after each epoch
those variable performs well on the training and testing set
'''
step_size = len(datas)
epochs = 200
pbar = tqdm(range(epochs))
step = 0
lr = 0.2
for epoch in pbar:
  for step in range(step_size):
    train(model,pocket,datas,labels,step,lr = lr)
    pbar.set_description('Epoch %2d:'%
          (epoch))
  lr = lr*0.99
  
#after training apply the weight tensor and the bias tensor in the pocket to the model
model.W = pocket.W
model.B = pocket.B

In [0]:
tf.print(pocket.W)
tf.print(pocket.B)
'''weights and bias in the report to apply just uncomment following codes'''
# previous = np.asarray([[0.98,4.55372334,-9.00216866],[1.54,-5.41183615,-9.5839529],[-2.24,2.86118317,12.7408953],[-1.17999983,-6.40912533,10.7291946]])
# model.W = previous
# model.B = previous

In [11]:
'''if using colab, use this function to update testing data set if run on local machine just ignore this function
if upload fails just give it another try
'''
uploaded = files.upload()

Saving iris_test.txt to iris_test.txt


In [0]:
'''data processing part extract testing dataset
Variables:
  datas_test:numpy.array of numpy.array
  labels_test:numpy.array of array
'''
datas_test = []
labels_test = []
datas_test_origin = []
with open("iris_test.txt") as f:
  for line in f:
    data = line.split(",")
    datas_test_origin.append(data[0:-1])
    datas_test.append(np.reshape(np.asarray(data[0:-1]),(4,1)))
    label_text = data[-1].replace("\n","")
    labels_test.append([[0,1][label_text=="Iris-setosa"],[0,1][label_text=="Iris-versicolor"],[0,1][label_text=="Iris-virginica"]])

datas_test = np.asarray(datas_test,dtype='float32')
labels_test = np.asarray(labels_test)

In [0]:
'''get the predicted output from the model and write the file
If download fails just give it another try
labels_prd_train:list
            used for confusion matrix
labels_act_train:list
            used for confusion matrix
labels_prd:list
            used for confusion matrix
labels_act:list
            used for confusion matrix
'''
labels_prd_train = []
labels_act_train = []
labels_prd = []
labels_act = []
with open("result.csv","w+") as f:
  writer = csv.writer(f)
  writer.writerow(["Sepal length"]+["Sepal width"]+["Petal length"]+["Petal width"]+["Actual class label"]+["Predicted class label"])
  for i in range(len(datas_test)):
    value = model(datas_test[i])
    label_act = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(labels_test[i]) else 0 for j in range(3)])]
    label_prd = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(value) else 0 for j in range(3)])]
    labels_act.append(label_act)
    labels_prd.append(label_prd)
    writer.writerow(datas_test_origin[i]+[label_act]+[label_prd])
labels_act = np.asarray(labels_act)
labels_prd = np.asarray(labels_prd)

for i in range(len(datas)):
  value = model(datas[i])
  label_act_train = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(labels[i]) else 0 for j in range(3)])]
  label_prd_train = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(value) else 0 for j in range(3)])]
  labels_act_train.append(label_act_train)
  labels_prd_train.append(label_prd_train)
labels_act_train = np.asarray(labels_act_train)
labels_prd_train = np.asarray(labels_prd_train)

In [0]:
'''after all of those download the csv file if fails just give it another try'''
files.download("result.csv")

In [0]:
def precision_recall_martix(act,prd):
  index_dict={"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2}
  matrix = np.array([[0,0,0],[0,0,0],[0,0,0]])
  for i in range(len(act)):
    matrix[index_dict[act[i]]][index_dict[prd[i]]] += 1
  print("---------------------confusion matrix-----------------------------")
  print(matrix)
  print("---------------------precision recall-----------------------------")
  presion_recall = np.array([[0,0],[0,0],[0,0]],dtype="float32")
  for i in range(3):
    presion_recall[i][0] = matrix[i][i]/(matrix[0][i]+matrix[1][i]+matrix[2][i])
    presion_recall[i][1] = matrix[i][i]/(len(act)/3)
  print(presion_recall)
#   
#   for i in range(3):
#     presion_recall[i][0] = matrix_pr[i][i]
#     for j in range(3):
#       if i != j:
#          presion_recall[i][1] += matrix_pr[i][j]
#   for i in range(3):
#     presion_recall[i][1] /= matrix_pr[i][i]
#   print( presion_recall)
  

In [144]:
print("-----------------------------training-------------------------------")
precision_recall_martix(labels_act_train,labels_prd_train)
print("-----------------------------testing-------------------------------")
precision_recall_martix(labels_act,labels_prd)

-----------------------------training-------------------------------
---------------------confusion matrix-----------------------------
[[38  2  0]
 [ 0 40  0]
 [ 0 26 14]]
---------------------precision recall-----------------------------
[[1.        0.95     ]
 [0.5882353 1.       ]
 [1.        0.35     ]]
-----------------------------testing-------------------------------
---------------------confusion matrix-----------------------------
[[10  0  0]
 [ 0 10  0]
 [ 0  7  3]]
---------------------precision recall-----------------------------
[[1.        1.       ]
 [0.5882353 1.       ]
 [1.        0.3      ]]


In [0]:
# '''precision-recall and confusin matrix by tools'''
# print("---------------------precision recall-----------------------------")
# print(sklearn.metrics.classification_report(labels_act,labels_prd))
# print("---------------------confusion matrix-----------------------------")
# print(sklearn.metrics.multilabel_confusion_matrix(labels_act,labels_prd))

In [0]:
'''the tool based model part
I select keras to build model with the same structure
but as the tool do not provide the pocket algorithm, 
I choose the as SGD optimaizer,CategoricalCrossentropy as loss function
And softmax as activation function
'''


'''build up the same 4-input 3-output model by keras'''
model_keras = tf.keras.Sequential()
model_keras.add(tf.keras.layers.Dense(3,input_shape=(4,),activation=tf.nn.softmax))

model_keras.compile(optimizer='sgd',loss = tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [0]:
'''make the shape of the inputs fit the outputs'''
datas_keras =[]
for item in datas:
  datas_keras.append(item.reshape(4,))
datas_keras = np.asarray(datas_keras)

In [96]:
'''train the model'''
model_keras.fit(x=datas_keras,y=labels,epochs=200)

Train on 120 samples
Epoch 1/200
120/120 [==============================] - 0s 3ms/sample - loss: 8.9137 - accuracy: 0.3333
Epoch 2/200
120/120 [==============================] - 0s 99us/sample - loss: 7.3012 - accuracy: 0.3333
Epoch 3/200
120/120 [==============================] - 0s 83us/sample - loss: 5.7194 - accuracy: 0.3333
Epoch 4/200
120/120 [==============================] - 0s 105us/sample - loss: 4.1539 - accuracy: 0.3333
Epoch 5/200
120/120 [==============================] - 0s 92us/sample - loss: 2.8807 - accuracy: 0.3333
Epoch 6/200
120/120 [==============================] - 0s 84us/sample - loss: 2.1291 - accuracy: 0.3333
Epoch 7/200
120/120 [==============================] - 0s 92us/sample - loss: 1.7795 - accuracy: 0.3250
Epoch 8/200
120/120 [==============================] - 0s 111us/sample - loss: 1.6045 - accuracy: 0.2667
Epoch 9/200
120/120 [==============================] - 0s 96us/sample - loss: 1.4997 - accuracy: 0.2333
Epoch 10/200
120/120 [====================

In [0]:
'''get the predicted output from the model
labels_prd_keras_train:list
            used for confusion matrix
labels_prd_keras:list
            used for confusion matrix
labels_act_keras:list
            used for confusion matrix
'''
labels_prd_keras_train = []
labels_prd_keras = []
labels_act_keras = []
for i in range(len(datas_test)):
  value = model_keras.predict(datas_test[i].reshape(1,4)).reshape(3,1)
  label_act_keras = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(labels_test[i]) else 0 for j in range(3)])]
  label_prd_keras = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(value) else 0 for j in range(3)])]
  labels_act_keras.append(label_act_keras)
  labels_prd_keras.append(label_prd_keras)
labels_prd_keras = np.asarray(labels_prd_keras)

for i in range(len(datas)):
  value = model_keras.predict(datas[i].reshape(1,4)).reshape(3,1)
  label_prd_keras_train = ["Iris-setosa","Iris-versicolor","Iris-virginica"][tf.argmax([1 if j == tf.math.argmax(value) else 0 for j in range(3)])]
  labels_prd_keras_train.append(label_prd_keras_train)
labels_prd_keras_train = np.asarray(labels_prd_keras_train)

In [141]:
print("-----------------------------training-------------------------------")
precision_recall_martix(labels_act_train,labels_prd_keras_train)
print("-----------------------------testing-------------------------------")
precision_recall_martix(labels_act_keras,labels_prd_keras)

-----------------------------training-------------------------------
---------------------confusion matrix-----------------------------
[[40  0  0]
 [ 0 34  6]
 [ 0  0 40]]
---------------------precision recall-----------------------------
[[1.        1.       ]
 [1.        0.85     ]
 [0.8695652 1.       ]]
-----------------------------testing-------------------------------
---------------------confusion matrix-----------------------------
[[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]
---------------------precision recall-----------------------------
[[1.  1. ]
 [0.9 0.9]
 [0.9 0.9]]


In [0]:
'''precision-recall and confusin matrix by tools'''
# print("---------------------precision recall-----------------------------")
# print(sklearn.metrics.classification_report(labels_act_keras,labels_prd_keras))
# print("---------------------confusion matrix-----------------------------")
# print(sklearn.metrics.multilabel_confusion_matrix(labels_act_keras,labels_prd_keras))